In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1 as ff1

ff1.Cache.enable_cache('cache')

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
def get_empty_dataframe():
    return pd.DataFrame(
            columns=[
                'LapNumber',
                'LapTime',
                'Compound',
                'TyreLife',
                'TrackStatus',
                'Stint',
                'DistanceToDriverAhead',
                'DriverAhead',
                ])

In [31]:
class NoTelemetryException(Exception):
    pass

def get_telemetry_at_start_of_lap(lap, telemetry):
    mask = (telemetry['Date'] > lap['LapStartDate']) & (telemetry['Date'] <= lap['LapStartDate'] + pd.Timedelta(seconds=1))
    rows = telemetry.loc[mask]
    if rows.empty:
        raise NoTelemetryException("No telemetry data found for lap " + str(lap['LapNumber']) + " of " + str(lap['Driver']) + " at " + str(lap['LapStartDate']))
    row = rows.iloc[0]
    telemetryInfo = row[['DriverAhead', 'DistanceToDriverAhead']]
    lapInfo = lap[['LapNumber', 'LapTime', 'Compound', 'TyreLife', 'Stint', 'TrackStatus']]
    telemetryInfo.rename("Telemetry", inplace=True)
    lapInfo.rename("Lap", inplace=True)
    merge = pd.concat([telemetryInfo, lapInfo])
    return merge

In [32]:
class NoLapException(Exception):
    pass

def get_laps_of_driver(driver, laps):
    driver_laps = laps.pick_driver(driver)
    if len(driver_laps['DriverNumber']) == 0:
        raise NoLapException("No laps for driver " + driver)
    driver_laps_telemetry = driver_laps.get_car_data()
    if len(driver_laps_telemetry) == 0:
        raise NoLapException("No telemetry for driver " + driver)
    try:
        driver_laps_telemetry = driver_laps_telemetry.add_driver_ahead()
        transformed_laps = []
        for index, row in driver_laps.iterrows():
            try:
                transformed_laps.append(get_telemetry_at_start_of_lap(row, driver_laps_telemetry))
            except NoTelemetryException as e:
                print(e)
    except ValueError as e:
        print("error : ", e)
        print(driver_laps_telemetry)
        return get_empty_dataframe()
    return pd.DataFrame(transformed_laps)

In [33]:
import os

def get_season_data(year, save_all_races=True):
    schedule = ff1.get_event_schedule(year, include_testing=False)
    
    path = 'data/' + str(year)
    # Create a directory for the year if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    df_season = get_empty_dataframe()
    for index, event in schedule.iterrows():
        if year == 2018 and event['RoundNumber'] < 3:
            continue
        print("Processing race round - ", event['RoundNumber'])
        race = event.get_race()
        race.load()
        df_event = get_empty_dataframe()
        for driver in race.drivers:
            print("     Processing driver - ", driver)
            try:
                df_event = pd.concat([df_event, get_laps_of_driver(driver, race.laps)], axis=0)
            except NoLapException as e:
                print(e)
        
        if save_all_races:
            # Save it to a csv file
            df_event.to_csv(path + '/' + event.EventName.replace(' ', '_').lower() + '.csv', index=False)
        df_season = pd.concat([df_season, df_event], axis=0)

    # Save the data for the whole season
    df_season.to_csv(path + '/season.csv', index=False)
    return df_season

In [36]:
get_season_data(2018)

core           INFO 	Loading data for Chinese Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data


Processing race round -  3


api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['3', '77', '7', '44', '33', '27', '14', '5', '55', '20', '31', '11', '2', '18', '35', '9', '8', '10', '16', '28']


     Processing driver -  3
     Processing driver -  77
     Processing driver -  7
     Processing driver -  44
     Processing driver -  33
     Processing driver -  27
     Processing driver -  14
     Processing driver -  5
     Processing driver -  55
     Processing driver -  20
     Processing driver -  31
     Processing driver -  11
     Processing driver -  2
     Processing driver -  18
     Processing driver -  35
     Processing driver -  9
     Processing driver -  8
     Processing driver -  10
     Processing driver -  16
     Processing driver -  28


core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v2.3.0]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!


...

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.3.0]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!


Processing race round -  21


api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for session_status_data. Loading data...
api            INFO 	Fetching session status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for track_status_data. Loading data...
api            INFO 	Fetching track status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for car_data. Loading data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Data 

     Processing driver -  44
     Processing driver -  5
     Processing driver -  33
     Processing driver -  3
     Processing driver -  77
     Processing driver -  55
     Processing driver -  16
     Processing driver -  11
     Processing driver -  8
     Processing driver -  20
     Processing driver -  14
     Processing driver -  28
     Processing driver -  18
     Processing driver -  2
     Processing driver -  35
     Processing driver -  10
     Processing driver -  31
     Processing driver -  9
     Processing driver -  7
     Processing driver -  27


,LapNumber,LapTime,Compound,TyreLife,TrackStatus,Stint,DistanceToDriverAhead,DriverAhead
0,1,NaT,NaN,NaN,1,NaN,NaN,
1,2,0 days 00:01:40.480000,ULTRASOFT,4.0,1,1.0,50.311111,44
2,3,0 days 00:01:40.059000,ULTRASOFT,5.0,1,1.0,107.520000,44
3,4,0 days 00:01:40.085000,ULTRASOFT,6.0,1,1.0,159.740000,44
4,5,0 days 00:01:39.715000,ULTRASOFT,7.0,1,1.0,191.015000,44
...,...,...,...,...,...,...,...,...
3,4.0,NaT,ULTRASOFT,8.0,4,1,36.915833,5
4,5.0,0 days 00:01:45.198000,ULTRASOFT,9.0,1,1,38.175278,5
5,6.0,0 days 00:01:55.382000,ULTRASOFT,10.0,1,1,74.178333,5
6,7.0,NaT,ULTRASOFT,11.0,267,1,14.793056,31


In [80]:
for year in range(2018, 2022):
    get_season_data(year)

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


Processing race round -  1


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']


     Processing driver -  5


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`